In [1]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.model.supervised import Supervised
from nlptools.utils import Config

cfg = Config("../config/cornell_gru.yml")
model = Supervised.build(cfg)

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


{'encoder': {'vocab_size': 30522, 'hidden_size': 768, 'num_hidden_layers': 3, 'bidirectional': True}, 'decoder': {'num_attention_heads': 8, 'num_hidden_layers': 3, 'shared_embed': True, 'max_seq_len': 20, 'attention': True}}


2019-05-20 14:28:08,550 | INFO: Optimizer: adam with learning_rate: 0.0001


In [2]:
dialog = iter(model.generator).next()
dialog.to("cuda:0")
response = dialog["response_cornell"].data
response_mask = dialog["response_mask_cornell"].data
utterance = dialog["utterance"].data
utterance_mask = dialog["utterance_mask"].data
sentiment = dialog["sentiment"].data
print(utterance, utterance.shape)
print(utterance_mask, utterance_mask.shape)
print(response, response.shape)
print(response_mask, response_mask.shape)
print(sentiment[:10, :], sentiment.shape)

tensor([[  101,  2748,  1012,  ...,     0,     0,     0],
        [  101, 17012,  2213,  ...,     0,     0,     0],
        [  101,  1045,  2572,  ...,     0,     0,     0],
        ...,
        [  101,  2021,  1996,  ...,     0,     0,     0],
        [  101,  2821,  2017,  ...,   102,     0,     0],
        [  101,  2008,  2003,  ...,     0,     0,     0]], device='cuda:0') torch.Size([88, 20])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0') torch.Size([88, 20])
tensor([[ 101, 2008, 2003,  ...,    0,    0,    0],
        [ 101, 2054, 2106,  ...,    0,    0,    0],
        [ 101, 2821, 1010,  ...,    0,    0,    0],
        ...,
        [ 101, 2053, 1012,  ..., 1012, 1012,  102],
        [ 101, 2017, 2024,  ...,    0,    0,    0],
        [ 101, 2748, 1010,  ...,    0,    0,    0]], device='cuda:0') t

In [3]:
encoder_out, encoder_hidden = model.skill.model.dialog_embedding(utterance, utterance_mask, sentiment)
prev_output = response.data[:, :-1]
target_output = response.data[:, 1:]
print(encoder_out.shape)
print(prev_output.shape)
print(target_output.shape)

torch.Size([88, 20, 768])
torch.Size([88, 19])
torch.Size([88, 19])


In [ ]:
for parameter in model.skill.model.decoder.fc3.parameters():
    print(parameter.requires_grad)

In [ ]:
target_output = target_output.unsqueeze(-1).contiguous().view(-1)
target_output.shape

In [29]:
#import nlptools.zoo.encoders.transformer
import nlptools.zoo.encoders.gru
import nlptools.zoo.encoders.attention
import importlib
#importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(nlptools.zoo.encoders.gru)
importlib.reload(nlptools.zoo.encoders.attention)
#from nlptools.zoo.encoders.transformer import TransformerDecoder, TransformerEncoder
from nlptools.zoo.encoders.gru import GRUDecoder

embedding = model.skill.model.encoder.embedding
decoder_hidden_layers = 1
decoder_attention_heads = 3
decoder_hidden_size = 2048
dropout = 0
# decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout=dropout)
decoder = GRUDecoder(embedding, decoder_hidden_layers, decoder_hidden_size, dropout=dropout)
decoder = decoder.to("cuda:0")
decoder

GRUDecoder(
  (word_embedding): Embedding(30522, 768)
  (gru): GRU(768, 768, batch_first=True)
  (attn): Linear(in_features=1536, out_features=20, bias=True)
  (attn_combine): Linear(in_features=1536, out_features=768, bias=True)
  (dropout): Dropout(p=0)
  (output_linear): Linear(in_features=768, out_features=768, bias=False)
)

In [30]:

output = decoder(prev_output, encoder_out, utterance_mask, encoder_hidden)
output.shape

torch.Size([88, 19, 30522])

In [ ]:
decoder